In [0]:
import os
import torch
import torchvision
import numpy as np
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image




In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 50
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [0]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5,),
                                     std=(0.5,))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)



0it [00:00, ?it/s]

100%|█████████▉| 9904128/9912422 [01:00<00:00, 131367.75it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



0it [00:00, ?it/s]
 57%|█████▋    | 16384/28881 [00:00<00:00, 117983.07it/s]
32768it [00:00, 226030.46it/s]                           
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100%|█████████▉| 1646592/1648877 [00:09<00:00, 165982.82it/s]

0it [00:00, ?it/s]

8192it [00:00, 115236.17it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw
Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


In [0]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1))

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Optimizers
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))



In [0]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    
    
# Loss weight for gradient penalty
lambda_gp = 10


def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = torch.Tensor(np.random.random((real_samples.size(0),1))).to(device)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.autograd.Variable(torch.Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False).to(device)
    # Get gradient w.r.t. interpolates
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty
  



In [0]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        for di in range(5):
          z = torch.randn(batch_size, latent_size).to(device)
          fake_images = G(z)
          D_real = D(images)
          D_fake = D(fake_images)
          
          # Gradient penalty
          gradient_penalty = compute_gradient_penalty(D, images.data, fake_images.data)
        
          # Backprop and optimize
          d_loss = -(torch.mean(D_real) - torch.mean(D_fake)) + lambda_gp * gradient_penalty
          
          reset_grad()
          d_loss.backward()
          d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        D_fake = D(fake_images)
        
        g_loss = -torch.mean(D_fake)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          D_real.mean().item(), D_fake.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))



Epoch [0/50], Step [200/600], d_loss: -2.0455, g_loss: -1.2089, D(x): 3.70, D(G(z)): 1.21
Epoch [0/50], Step [400/600], d_loss: -2.4814, g_loss: -0.4019, D(x): 2.66, D(G(z)): 0.40
Epoch [0/50], Step [600/600], d_loss: -2.2230, g_loss: -0.6056, D(x): 3.38, D(G(z)): 0.61
Epoch [1/50], Step [200/600], d_loss: -2.4893, g_loss: -0.9647, D(x): 3.58, D(G(z)): 0.96
Epoch [1/50], Step [400/600], d_loss: -2.4284, g_loss: -0.6332, D(x): 3.11, D(G(z)): 0.63
Epoch [1/50], Step [600/600], d_loss: -2.1089, g_loss: -0.6519, D(x): 2.74, D(G(z)): 0.65
Epoch [2/50], Step [200/600], d_loss: -2.1764, g_loss: -1.1448, D(x): 3.40, D(G(z)): 1.14
Epoch [2/50], Step [400/600], d_loss: -2.4214, g_loss: -0.5016, D(x): 2.89, D(G(z)): 0.50
Epoch [2/50], Step [600/600], d_loss: -2.1915, g_loss: -1.4140, D(x): 3.78, D(G(z)): 1.41
Epoch [3/50], Step [200/600], d_loss: -2.0508, g_loss: -0.7361, D(x): 3.10, D(G(z)): 0.74
Epoch [3/50], Step [400/600], d_loss: -2.1734, g_loss: -0.1380, D(x): 2.73, D(G(z)): 0.14
Epoch [3/5

In [0]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')